# Utilities

> Utility functions and classes used by custom transforms.

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Tuple
import numpy as np

In [ ]:
#| export
def _find_largest_rectangle_coords(binary_mask: np.ndarray
                                  ) -> Tuple[slice, slice]:
    """
    Use a row-by-row 'largest rectangle in histogram' approach to find the largest
    rectangle of True cells in a 2D binary numpy array.
    
    Returns:
        (row_slice, col_slice) indicating the largest rectangle found.
    """
    rows, cols = binary_mask.shape
    heights = np.zeros((rows, cols), dtype=int)
    
    # Build up "histogram" heights for each column per row
    for col_idx in range(cols):
        for row_idx in range(rows):
            if binary_mask[row_idx, col_idx]:
                heights[row_idx, col_idx] = (
                    heights[row_idx - 1, col_idx] + 1 if row_idx > 0 else 1
                )

    max_area = 0
    best_rectangle = (slice(0, 1), slice(0, 1))  # A default small rectangle

    # For each row, find the largest rectangle in histogram
    for row_idx in range(rows):
        current_area, (top, bottom, left, right) = _largest_rectangle_in_histogram(
            heights[row_idx]
        )
        # current_area is for the single "row" histogram, but we must offset vertically
        # by row_idx based on how tall the histogram is
        if current_area > 0:
            # bottom row is row_idx
            # top row is row_idx - heights for the chosen rectangle + 1
            # we need to figure out the actual top row from the histogram table
            rect_height = heights[row_idx, left:right].min()
            actual_top = row_idx - rect_height + 1
            actual_bottom = row_idx + 1  # slices go one past the end
            
            if current_area > max_area:
                max_area = current_area
                best_rectangle = (
                    slice(actual_top, actual_bottom),
                    slice(left, right)
                )

    return best_rectangle

In [ ]:
#| export
def _largest_rectangle_in_histogram(row_heights: np.ndarray
                                   ) -> Tuple[int, Tuple[int,int,int,int]]:
    """
    Given a 1D array 'row_heights', returns the area of the largest rectangle in this
    histogram plus the (top, bottom, left, right) indices.
    
    Note: In this context, 'top' and 'bottom' are placeholders, because the actual
    vertical extent is determined later using the 2D 'heights' array.
    """
    stack = []  # will hold (start_index, height)
    max_area = 0
    # (top, bottom, left, right) in histogram terms only
    best_coords = (0, 0, 0, 0)

    for idx in range(len(row_heights) + 1):
        # height is 0 if beyond the last column, to flush out the stack
        current_height = row_heights[idx] if idx < len(row_heights) else 0
        start_index = idx

        while stack and stack[-1][1] > current_height:
            # Pop the stack
            prev_index, prev_height = stack.pop()
            area = prev_height * (idx - prev_index)
            if area > max_area:
                max_area = area
                # left = prev_index, right = idx
                best_coords = (0, prev_height, prev_index, idx)
            start_index = prev_index

        if current_height > 0:
            stack.append((start_index, current_height))

    return max_area, best_coords

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()